In [1]:
%matplotlib qt
import matplotlib.pyplot as plt
import hyperspy.api as hs
import pyxem as pxm
import numpy as np
from pathlib import Path
from diffsims.utils.sim_utils import get_electron_wavelength

In [2]:
import numpy as np
a = 4.04
hkl = np.array([8, 0, 0])
g = np.sqrt(np.sum(hkl**2/a**2))

In [3]:
print(g)

1.9801980198019802


In [4]:
#datapath = Path(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Data\2021_10_06_2xxx_24h_250C\Preprocessed_data\SPED_600x600x12_10x10_4p63x4p63_1deg_100Hz_CL12cm_NBD_alpha5_spot1p3_preprocessed.hspy')
datapath = Path(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Data\2022_06_20_ESTEEM_prep_2xxx_24h_250C\SPED_512x512x6_5x5_2p315x2p315_NBD_alpha5_spot1p0nm_CL12cm_1deg_100Hz_preprocessed.hspy')
signal = hs.load(datapath, lazy=True)

In [5]:
signal.metadata.Preprocessing.Masks

└── Diffraction
    ├── cutoff = <LazySignal2D, title: <0.7001057239470767 $A^{-1}$ mask, dimensions: (|128, 128)>
    ├── direct_beam = <LazySignal2D, title: >0.004612922335229383 $A^{-1}$ mask, dimensions: (|128, 128)>
    └── reflections = <LazySignal2D, title: Reflection mask, dimensions: (|128, 128)>

In [6]:
mask = masks[0]

NameError: name 'masks' is not defined

In [ ]:
mask[1]

In [18]:
masks = [mask[1] for mask in signal.metadata.Preprocessing.Masks.Diffraction]
print(masks)

[<LazySignal2D, title: <0.7001057239470767 $A^{-1}$ mask, dimensions: (|128, 128)>, <LazySignal2D, title: >0.15949382751461458 $A^{-1}$ mask, dimensions: (|128, 128)>, <LazySignal2D, title: Reflection mask, dimensions: (|128, 128)>]


In [30]:
signal.metadata.Preprocessing.Masks.diffraction

AttributeError: diffraction

In [31]:
fig, ax = plt.subplots()
ax.imshow(diffmask)

In [24]:
diffmask

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [7]:
#datapath = Path(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Data\2021_10_06_2xxx_24h_250C\Site2\SPED_600x600x12_10x10_4p63x4p63_1deg_100Hz_CL12cm_NBD_alpha5_spot1p3.hspy')
datapath = Path(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Data\2022_06_20_ESTEEM_prep_2xxx_24h_250C\SPED_512x512x6_5x5_2p315x2p315_NBD_alpha5_spot1p0nm_CL12cm_1deg_100Hz.hspy')
raw_signal = hs.load(datapath, lazy=True)
preprocessed_signal = hs.load(datapath.with_name(f'{datapath.stem}_preprocessed.hspy'), lazy=True)

In [8]:
shifts = preprocessed_signal.metadata.Preprocessing.Centering.Shifts
raw_signal = raw_signal.inav[0:512, 0:512]
raw_signal.shift_diffraction(shifts.isig[0], shifts.isig[1], inplace=True)

In [9]:
mean = raw_signal.mean(axis=[0, 1])
mean.compute()

[########################################] | 100% Completed |  5min 13.2s


In [10]:
mean.plot(norm='symlog')
lines = {i: hs.roi.Line2DROI(0, 0, 0.1) for i in range(2)}
[lines[line].add_widget(mean) for line in lines]

In [11]:
sep='\n'
print(sep.join([f'Line {line}: ({lines[line].x1}, {lines[line].y1}) <-> ({lines[line].x2}, {lines[line].y2}' for line in lines]))

Line 0: (0.5723, -1.0767) <-> (-0.5432, 1.0282
Line 1: (-1.067, -0.5820000000000001) <-> (1.0573000000000001, 0.5723


In [12]:
profiles = {line: lines[line](mean) for line in lines}
[profiles[profile].plot(norm='log') for profile in profiles]
points = {profile: [hs.roi.Point1DROI(0), hs.roi.Point1DROI(0)] for profile in profiles}
[[point.add_widget(profiles[profile]) for point in points[profile]] for profile in profiles]

[[<hyperspy.drawing._widgets.vertical_line.VerticalLineWidget at 0x2230039d4e0>,

In [13]:
sep='\n'
print(sep.join([f'Profile {profile} peaks: {[points[profile][i].value for i in range(len(points[profile]))]}' for profile in profiles]))

Profile 0 peaks: [0.2025364257716686, 2.218256091784942]
Profile 1 peaks: [0.1926418867366388, 2.2250137918081783]


In [14]:
len(points[0])

2

In [15]:
lengths = {profile: (np.max([points[profile][i].value for i in range(len(points[profile]))]) - np.min([points[profile][i].value for i in range(len(points[profile]))])) / profiles[profile].axes_manager[-1].scale for profile in profiles}

sep = '\n'
print(sep.join([f'Profile {profile} peak to peak distance: {lengths[profile]}' for profile in profiles]))

hkl_values = {profile: np.array([8, 0, 0]) for profile in profiles}
a = 4.04
g_values = {profile: np.sqrt(np.sum((hkl_values[profile]/a)**2)) for profile in profiles}
scales = {profile: g_values[profile] / lengths[profile] for profile in profiles}
sep='\n'
print(sep.join([f'Profile {profile} scale: {scales[profile]}' for profile in profiles]))

print(f'Mean calibration: {np.mean(list(scales.values()))}')
print(f'std calibration: {np.std(list(scales.values()))}')

scale = np.round(np.mean(list(scales.values())), decimals=5)
print(f'Using mean calibration as calibration: {scale:.05f}')

binning_factors = [raw_signal.axes_manager.signal_shape[i]/preprocessed_signal.axes_manager.signal_shape[i] for i in range(2)]
print(f'Preprocessed signal is binned with scales {binning_factors}')
rescaled_scales = [scale*binning_factor for binning_factor in binning_factors]
print(f'Preprocessed signal has calibration {rescaled_scales}')
for i, scale in enumerate(rescaled_scales):
    preprocessed_signal.axes_manager[preprocessed_signal.axes_manager.navigation_dimension+i].scale = scale

Profile 0 peak to peak distance: 209.00000000000003
Profile 1 peak to peak distance: 211.0
Profile 0 scale: 0.009474631673693683
Profile 1 scale: 0.009384824738398011
Mean calibration: 0.009429728206045847
std calibration: 4.490346764783604e-05
Using mean calibration as calibration: 0.00943
Preprocessed signal is binned with scales [2.0, 2.0]
Preprocessed signal has calibration [0.01886, 0.01886]


In [16]:
preprocessed_signal.axes_manager

Navigation axis name,size,index,offset,scale,units
x,512,0,0.0,2.315,nm
y,512,0,0.0,2.315,nm
Signal axis name,size,,offset,scale,units
kx,128,,-1.2138,0.01886,$A^{-1}$
ky,128,,-1.2138,0.01886,$A^{-1}$


In [17]:
from skimage.feature import blob_log

## Mask reflections

In [19]:
signal.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── Acquisition_parameters
│       │   ├── alpha = 5
│       │   ├── beam_energy = 200.0
│       │   ├── camera = Merlin
│       │   ├── camera_length = 12.0
│       │   ├── convergence_angle = None
│       │   ├── exposure_time = None
│       │   ├── microscope = 2100F
│       │   ├── mode = NBD
│       │   ├── notes = None
│       │   ├── operator = Emil Christiansen
│       │   ├── rocking_angle = 1.0
│       │   ├── rocking_frequency = 100.0
│       │   ├── scan_rotation = None
│       │   ├── specimen = 2xxx 250C 24h
│       │   └── spotsize = 1.0
│       ├── Detector
│       │   └── Diffraction
│       │       └── camera_length = 22.586
│       ├── beam_energy = 200.0
│       ├── rocking_angle = 1.0
│       └── rocking_frequency = 100.0
├── General
│   ├── FileIO
│   │   ├── 0
│   │   │   ├── hyperspy_version = 1.7.0.dev0
│   │   │   ├── io_plugin = hyperspy.io_plugins.hspy
│   │   │   ├── operation = save
│   │   │   └── timestamp = 2022-07-04T15:35:46.330928+02:00
│   │   ├── 1
│   │   │   ├── hyperspy_version = 1.7.0.dev0
│   │   │   ├── io_plugin = hyperspy.io_plugins.hspy
│   │   │   ├── operation = load
│   │   │   └── timestamp = 2022-07-05T11:58:51.393823+02:00
│   │   ├── 2
│   │   │   ├── hyperspy_version = 1.7.0.dev0
│   │   │   ├── io_plugin = hyperspy.io_plugins.hspy
│   │   │   ├── operation = save
│   │   │   └── timestamp = 2022-07-05T12:04:18.678231+02:00
│   │   └── 3
│   │       ├── hyperspy_version = 1.7.0
│   │       ├── io_plugin = hyperspy.io_plugins.hspy
│   │       ├── operation = load
│   │       └── timestamp = 2022-07-05T14:31:36.502478+02:00
│   └── title = 
├── HDR
│   ├── Acquisition Type (Normal, Th_scan, Config) = Normal
│   ├── Active Counters = Counter 0
│   ├── Assembly Size (NX1, 2X2) = 1x1
│   ├── Chip ID = W559_G11, - , - , -
│   ├── Chip Mode  (SPM, CSM, CM, CSCM) = SPM
│   ├── Chip Type (Medipix 3.0, Medipix 3.1, Medipix 3RX) = Medipix 3RX
│   ├── Counter Depth (number) = 12
│   ├── DAC File = c:\MERLIN_Quad_Config\W559_G11\W559_G11_SPM.dacs,,,
│   ├── DACs = 068,511,000,000,000,000,000,000,100,255,100,125,100,100,080,100,090,030,128,004,255,126,128,174,172,511,511; ; ;
│   ├── Dead Time File = Dummy (C:\<NUL>\)
│   ├── Flat Field File = None
│   ├── Frames in Acquisition (Number) = 262144
│   ├── Frames per Trigger (Number) = 1
│   ├── Gain = SLGM
│   ├── Gap Fill Mode = None
│   ├── Humidity (%) = Board Humidity 0.000000
│   ├── Medipix Clock (MHz) = 160MHz
│   ├── Readout System = Merlin Quad
│   ├── Sensor Bias (V) = 120 V
│   ├── Sensor Polarity (Positive, Negative) = Positive
│   ├── Software Version = 1.0.0.122
│   ├── Temperature (C) = Board Temp 0.000000 Deg C
│   ├── Thresholds (keV) = 4.000000E+1,5.110000E+2,0.000000E+0,0.000000E+0,0.000000E+0,0.000000E+0,0.000000E+0,0.000000E+0
│   ├── Time and Date Stamp (day, mnth, yr, hr, min, s) = 20/06/2022 12:53:12
│   ├── Trigger Start (Positive, Negative, Internal) = Rising Edge
│   ├── Trigger Stop (Positive, Negative, Internal) = Internal
│   └── bpc File = c:\MERLIN_Quad_Config\W559_G11\W559_G11_SPM.bpc,,,
├── Preprocessing
│   ├── Centering
│   │   ├── COM = <LazyDPCSignal2D, title: , dimensions: (2|512, 512)>
│   │   ├── COM_mask
│   │   │   ├── r = 12.5
│   │   │   ├── x = 125
│   │   │   └── y = 125
│   │   ├── Shifts = <LazyBeamShift, title: , dimensions: (512, 512|2)>
│   │   └── shift_estimate_mask = <LazySignal, title: , dimensions: (512, 512|)>
│   ├── Masks
│   │   └── Diffraction
│   │       ├── cutoff = <LazySignal2D, title: <0.7001057239470767 $A^{-1}$ mask, dimensions: (|128, 128)>
│   │       ├── direct_beam = <LazySignal2D, title: >0.004612922335229383 $A^{-1}$ mask, dimensions: (|128, 128)>
│   │       └── reflections = <LazySignal2D, title: Reflection mask, dimensions: (|128, 128)>
│   ├── Maximums = <LazyElectronDiffraction2D, title: , dimensions: (|128, 128)>
│   └── VBF = <LazySignal2D, title: Integrated intensity, dimensions: (|512, 512)>
└── Signal
    └──

In [5]:
binned = raw_signal.rebin(scale=(1, 1, 2, 2))

In [20]:
data = signal.max(axis=[0, 1, 2, 3]).data
data.compute()
print(data)

KeyboardInterrupt: 

In [ ]:
print(data)

In [ ]:
print('test')

In [21]:
signal.axes_manager

Navigation axis name,size,index,offset,scale,units
x,512,0,0.0,2.315,nm
y,512,0,0.0,2.315,nm
Signal axis name,size,,offset,scale,units
kx,128,,-1.2138,0.01904,$A^{-1}$
ky,128,,-1.2138,0.01904,$A^{-1}$


In [22]:
#image = (signal*(2**16-1)).mean(axis=[0, 1])
#image = binned.mean(axis=[0, 1])
image = preprocessed_signal.mean(axis=[0, 1])
image.compute()

[########################################] | 100% Completed |  2min 46.9s


In [23]:
im = image.deepcopy()

In [40]:
image.change_dtype('uint16')

In [55]:
2**16

65536

In [52]:
image.max(axis=[0, 1]).data

array([65106.0457243])

In [25]:
a = 4.04
minimum_r = 8 #
cutoff_hkl = np.array([2, 2, 0]) #Make a mask with cutoff at a given HKL
cuton_mrad = 4 #Make a mask that cutsoff everything up a certain mrad

#Set up mask arrays
nx, ny = image.axes_manager.signal_shape
mask = np.zeros((nx, ny), dtype=bool)
direct_beam_mask = np.zeros((nx, ny), dtype=bool)
cutoff_mask = np.zeros((nx, ny), dtype=bool)

#Setting cutoffs
cutoff_g = np.sqrt(np.sum(cutoff_hkl**2 / a**2))
cuton_k = cuton_mrad / 1000 / get_electron_wavelength(image.metadata.Acquisition_instrument.TEM.beam_energy/1000)
print(f'Minimum scattering vector: {cuton_k} {image.axes_manager[0].units}\nMaximum scattering vector: {cutoff_g} {image.axes_manager[0].units}')
X, Y = np.meshgrid(image.axes_manager[0].axis, image.axes_manager[1].axis)
#Set outer cutoff
R = np.sqrt(X**2 + Y**2)
cutoff_mask[R>=cutoff_g] = True
#Set inner cutoff
R = np.sqrt(X**2 + Y**2)
direct_beam_mask[R<=cuton_k] = True

#Mask reflections
blob_kwargs = {
    'min_sigma': 1,
    'max_sigma': 15,
    'num_sigma': 100,
    'overlap': 0,
    'threshold': 1.4E1,
}

# Look for blobs (reflections)
blobs = blob_log(image.data, **blob_kwargs)
print(len(blobs))
xs, ys = np.arange(0, nx), np.arange(0, ny)
X, Y = np.meshgrid(xs, ys)
for blob in blobs:
    y, x, r = blob  # x and y axes are flipped in hyperspy compared to numpy
    r = np.sqrt(2) * r  # Scale blob radius to appear more like a real radius
    r = max([minimum_r, r])  # Make sure that the radius is at least the specified minimum radius
    R = np.sqrt((X - x) ** 2 + (Y - y) ** 2)
    mask[R < r] = True

fig, axes = plt.subplots(nrows=1, ncols=6, sharex=True, sharey=True)
axes[0].imshow(image.data)
axes[1].imshow(mask)
axes[2].imshow(direct_beam_mask)
axes[3].imshow(cutoff_mask)
axes[4].imshow((image.data * ~(mask + direct_beam_mask))**0.256)
axes[5].imshow((image.data * ~cutoff_mask)**0.256)

Minimum scattering vector: 0.004612922335229383 $A^{-1}$
Maximum scattering vector: 0.7001057239470767 $A^{-1}$
25
